<a href="https://colab.research.google.com/github/n1gek/Image-Classification-Model/blob/main/mymodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
!pip install tensorflow_addons
import tensorflow_addons as tfa
import tensorflow_addons.activations as Ac


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import *
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras import activations as Ac
import h5py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 14.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
data_path = '/content/drive/My Drive/AIR/AIR/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data'
train_folder = 'train_folder'
images = os.listdir(os.path.join(data_path, train_folder))
len(images)


3334

In [ ]:
train = pd.read_csv(os.path.join(data_path, 'new_train'))
test = pd.read_csv(os.path.join(data_path, 'new_test'))
val = pd.read_csv(os.path.join(data_path, 'new_val'))
train.head()

,filename,Classes,Labels,family,manufacturer,variant,filepath
0,1025794.jpg,707-320,0,Boeing 707,Boeing,707-320,train_folder/1025794.jpg
1,1340192.jpg,707-320,0,Boeing 707,Boeing,707-320,train_folder/1340192.jpg
2,0056978.jpg,707-320,0,Boeing 707,Boeing,707-320,train_folder/0056978.jpg
3,0698580.jpg,707-320,0,Boeing 707,Boeing,707-320,train_folder/0698580.jpg
4,0450014.jpg,707-320,0,Boeing 707,Boeing,707-320,train_folder/0450014.jpg


In [ ]:
train['filepath'] = train['filepath'].apply(lambda x: os.path.join(data_path, x))
test['filepath'] = test['filepath'].apply(lambda x: os.path.join(data_path, x))
val['filepath'] = val['filepath'].apply(lambda x: os.path.join(data_path, x))


In [ ]:
train_paths = train['filepath']
test_paths = test['filepath']
val_paths = val['filepath']
train_paths.values

array(['/content/drive/My Drive/AIR/AIR/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/train_folder/1025794.jpg',
       '/content/drive/My Drive/AIR/AIR/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/train_folder/1340192.jpg',
       '/content/drive/My Drive/AIR/AIR/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/train_folder/0056978.jpg',
       ...,
       '/content/drive/My Drive/AIR/AIR/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/train_folder/0472681.jpg',
       '/content/drive/My Drive/AIR/AIR/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/train_folder/1597829.jpg',
       '/content/drive/My Drive/AIR/AIR/fgvc-aircraft-2013b/fgvc-aircraft-2013b/data/train_folder/0197891.jpg'],
      dtype=object)

In [ ]:
train_folder = 'train_folder'
def image_exists(filepath):
    full_path = os.path.join(data_path, train_folder, filepath)
    return os.path.isfile(full_path)

# Filter out rows with missing images
train_cleaned = train[train['filepath'].apply(image_exists)]

# Display the cleaned DataFrame
print("DataFrame after removing entries with missing images:")
print(train_cleaned.head())

# Optionally save the cleaned DataFrame to a new CSV file
train_cleaned.to_csv(os.path.join(data_path, 'train_cleaned.csv'), index=False)

NameError: name 'train' is not defined

In [ ]:
len(train_cleaned)

3334

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
train.head()

,filename,Classes,Labels,family,manufacturer,variant,filepath
0,1025794.jpg,707-320,0,Boeing 707,Boeing,707-320,/content/drive/My Drive/AIR/AIR/fgvc-aircraft-...
1,1340192.jpg,707-320,0,Boeing 707,Boeing,707-320,/content/drive/My Drive/AIR/AIR/fgvc-aircraft-...
2,0056978.jpg,707-320,0,Boeing 707,Boeing,707-320,/content/drive/My Drive/AIR/AIR/fgvc-aircraft-...
3,0698580.jpg,707-320,0,Boeing 707,Boeing,707-320,/content/drive/My Drive/AIR/AIR/fgvc-aircraft-...
4,0450014.jpg,707-320,0,Boeing 707,Boeing,707-320,/content/drive/My Drive/AIR/AIR/fgvc-aircraft-...


In [ ]:
train_labels = to_categorical(train.Labels)
train_labels.shape

(3334, 100)

In [ ]:
val_labels = to_categorical(val.Labels)
val_labels.shape

(3333, 100)

In [ ]:
def decode_image(filename, label=None, image_size=(299, 299)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)

    if label is None:
        return image
    else:
        return image, label

In [ ]:
def augment(image, label=None):
    image = tf.image.random_flip_left_right(image)

    if label is None:
        return image
    else:
        return image, label

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
batch_size = 16

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(AUTO)
)

val_dataset = (tf.data.Dataset
        .from_tensor_slices((val_paths, val_labels))
        .map(decode_image, num_parallel_calls=AUTO)
        .batch(batch_size)
)

test_dataset = (tf.data.Dataset
        .from_tensor_slices(test_paths)
        .map(decode_image, num_parallel_calls=AUTO)
        .batch(batch_size)
               )

In [ ]:
import tensorflow_addons as tfa

def Ranger(sync_period=6,
           slow_step_size=0.5,
           learning_rate=0.01,
           beta_1=0.9,
           beta_2=0.999,
           epsilon=1e-7,
           weight_decay=0.,
           amsgrad=False,
           sma_threshold=5.0,
           total_steps=0,
           warmup_proportion=0.1,
           min_lr=0.,
           name="Ranger"):
     # create RAdam optimizer
    inner = tfa.optimizers.RectifiedAdam(learning_rate, beta_1, beta_2, epsilon, weight_decay, amsgrad, sma_threshold, total_steps, warmup_proportion, min_lr, name)
    # feed RAdam optimizer into lookahead operation
    optim = tfa.optimizers.Lookahead(inner, sync_period, slow_step_size, name)
    return optim

In [ ]:
import numpy as np
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K

class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        self.scale_fn = scale_fn
        self.scale_mode = scale_mode
        self.clr_iterations = 0
        self.trn_iterations = 0
        self.history = {}

        # Initialize the scale function based on the mode
        if self.scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1 / (2. ** (x - 1))
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma ** x
        else:
            self.scale_fn = scale_fn

    def clr(self):
        cycle = np.floor(1 + self.clr_iterations / (2 * self.step_size))
        x = np.abs(self.clr_iterations / self.step_size - 2 * cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr - self.base_lr) * np.maximum(0, (1 - x)) * self.scale_fn(self.clr_iterations)

    def on_train_begin(self, logs=None):
        logs = logs or {}
        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        # Update learning rate
        K.set_value(self.model.optimizer.lr, self.clr())

        # Track the learning rate and other metrics
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)


In [ ]:
def mish(x):
    return x * K.tanh(Ac.softplus(x))

model = tf.keras.Sequential([
        DenseNet201(weights = 'imagenet',
                       include_top = False,
                       input_shape = (299, 299, 3)),

        GlobalAveragePooling2D(),
        Dense(1024, activation = mish),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation = mish),
        BatchNormalization(),
        Dropout(0.5),
        Dense(100, activation = 'softmax')
])
optimizer = SGD()
model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

model.summary()

74836368/74836368 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 9, 9, 1920)        18321984  
                                                                 
 global_average_pooling2d (  (None, 1920)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1024)              1967104   
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                         

In [ ]:
def Train_model(model, batch_size, EPOCHS):

    n_steps = train_labels.shape[0] // batch_size
    EPOCHS = EPOCHS

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
                                                     factor = 0.5,
                                                     patience = 3,
                                                     min_lr = 0.0001)

    clr = CyclicLR(base_lr = 0.001,
                   max_lr = 0.01,
                   step_size = 2000,
                   mode = 'triangular',)


    Model = model
    history = Model.fit(train_dataset,
                    steps_per_epoch = n_steps,
                    epochs = EPOCHS,
                    validation_data = val_dataset,
                    callbacks = [clr],
                    verbose = 1)

    return Model

In [ ]:
batch_size = 16
EPOCHS = 40

def check(x, y):
    if x == y:
        return 1
    else:
        return 0

print('Training')
model = Train_model(model, batch_size, EPOCHS)
pred = model.predict(test_dataset, verbose=1)

test['Prediction'] = pred.argmax(axis=1)
test['Score'] = test.apply(lambda x: check(x['Prediction'], x['Labels']), axis=1)
print('Test accuracy on iterations is ', 100 * test.Score.sum() / test.shape[0])

Training
Epoch 1/40
208/208 [==============================] - 476s 1s/step - loss: 5.7921 - accuracy: 0.0204 - val_loss: 4.5020 - val_accuracy: 0.0321
Epoch 2/40
208/208 [==============================] - 38s 181ms/step - loss: 4.7403 - accuracy: 0.0724 - val_loss: 3.6830 - val_accuracy: 0.1572
Epoch 3/40
208/208 [==============================] - 38s 182ms/step - loss: 3.7487 - accuracy: 0.1626 - val_loss: 2.9376 - val_accuracy: 0.2568
Epoch 4/40
208/208 [==============================] - 38s 181ms/step - loss: 2.8362 - accuracy: 0.3029 - val_loss: 2.3020 - val_accuracy: 0.4101
Epoch 5/40
208/208 [==============================] - 38s 182ms/step - loss: 2.2387 - accuracy: 0.4156 - val_loss: 1.9463 - val_accuracy: 0.4674
Epoch 6/40
208/208 [==============================] - 38s 181ms/step - loss: 1.6792 - accuracy: 0.5478 - val_loss: 1.7529 - val_accuracy: 0.5263
Epoch 7/40
208/208 [==============================] - 38s 181ms/step - loss: 1.3316 - accuracy: 0.6268 - val_loss: 1.7434 -

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(test.Labels, test.Prediction))

              precision    recall  f1-score   support

           0       0.74      0.70      0.72        33
           1       0.85      0.88      0.87        33
           2       0.93      0.74      0.82        34
           3       0.43      0.39      0.41        33
           4       0.53      0.55      0.54        33
           5       0.62      0.59      0.61        34
           6       0.66      0.82      0.73        33
           7       0.59      0.58      0.58        33
           8       0.59      0.47      0.52        34
           9       0.72      0.79      0.75        33
          10       0.57      0.36      0.44        33
          11       0.40      0.50      0.44        34
          12       0.63      0.67      0.65        33
          13       0.69      0.61      0.65        33
          14       0.82      0.68      0.74        34
          15       0.80      0.85      0.82        33
          16       0.59      0.70      0.64        33
          17       0.32    